In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
index_name= df[ df['Borough'] =='Not assigned'].index
df.drop(index_name, inplace=True)

In [4]:
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
df.head(10)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
result = df.groupby(['Postal Code','Borough'], sort=False).agg( ', '.join)

In [7]:
df_new=result.reset_index()
df_new.head(15)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
df_new.shape

(103, 3)

In [9]:
df_geo_coor = pd.read_csv("http://cocl.us/Geospatial_data")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_toronto = pd.merge(df_new, df_geo_coor)
# remove the "Postal Code" column
df_toronto.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [11]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
import requests
import numpy as np

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


In [12]:
map_scarborough = folium.Map(location=[43.78, -79.20], zoom_start=13)
map_scarborough

In [13]:
Scarborough = df_toronto.loc[df_toronto['Borough'] == "Scarborough"]
Scarborough.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
22,M1G,Scarborough,Woburn,43.770992,-79.216917
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [15]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [16]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto


In [17]:
CLIENT_ID = 'Y45O4KNOIIOH0OUJLXWFOK5NXWKIOTETR1AUQNNT11S5TEKH' # your Foursquare ID
CLIENT_SECRET = 'YLCIXAMIHWOBFXP3GB52RJKAIAVTNRVLJDB1OLHZBAMEI3MB'  # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y45O4KNOIIOH0OUJLXWFOK5NXWKIOTETR1AUQNNT11S5TEKH
CLIENT_SECRET:YLCIXAMIHWOBFXP3GB52RJKAIAVTNRVLJDB1OLHZBAMEI3MB


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        print(name, len(results))
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
scarborough_venues = getNearbyVenues(names=Scarborough['Postal Code'],
                                   latitudes=Scarborough['Latitude'],
                                   longitudes=Scarborough['Longitude']
                                  )

M1B
M1B 2
M1C
M1C 1
M1E
M1E 8
M1G
M1G 3
M1H
M1H 8
M1J
M1J 1
M1K
M1K 7
M1L
M1L 7
M1M
M1M 2
M1N
M1N 4
M1P
M1P 5
M1R
M1R 4
M1S
M1S 4
M1T
M1T 13
M1V
M1V 3
M1W
M1W 15
M1X
M1X 0


In [20]:
scarborough_venues.head()



,Postal Code,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M1B,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,M1B,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,M1C,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,M1E,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,M1E,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [21]:
len(scarborough_venues['Venue'].unique())

77

In [22]:
len(scarborough_venues['Venue Category'].unique())

52

In [23]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Postal Code'] = scarborough_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Postal Code,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Café,...,Playground,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Thai Restaurant,Train Station,Vietnamese Restaurant
0,M1B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M1B,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,M1C,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M1E,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M1E,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
scarborough_grouped = scarborough_onehot.groupby('Postal Code').mean().reset_index()
scarborough_grouped.head()

,Postal Code,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Café,...,Playground,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Thai Restaurant,Train Station,Vietnamese Restaurant
0,M1B,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.0,...,0.0,0.5,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0
1,M1C,0.0,0.000,0.0,0.000,0.000,1.0,0.000,0.0,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0
2,M1E,0.0,0.000,0.0,0.000,0.125,0.0,0.125,0.0,0.0,...,0.0,0.0,0.125,0.125,0.0,0.0,0.0,0.000,0.0,0.0
3,M1G,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.0
4,M1H,0.0,0.125,0.0,0.125,0.125,0.0,0.000,0.0,0.0,...,0.0,0.0,0.000,0.000,0.0,0.0,0.0,0.125,0.0,0.0


In [25]:
num_top_venues = 5

for hood in scarborough_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                   venue  freq
0             Print Shop   0.5
1   Fast Food Restaurant   0.5
2    American Restaurant   0.0
3                   Park   0.0
4  Korean BBQ Restaurant   0.0


----M1C----
                       venue  freq
0                        Bar   1.0
1        American Restaurant   0.0
2                  Pet Store   0.0
3      Korean BBQ Restaurant   0.0
4  Latin American Restaurant   0.0


----M1E----
               venue  freq
0       Intersection  0.12
1  Electronics Store  0.12
2               Bank  0.12
3     Breakfast Spot  0.12
4     Medical Center  0.12


----M1G----
                       venue  freq
0                Coffee Shop  0.67
1      Korean BBQ Restaurant  0.33
2        American Restaurant  0.00
3                  Pet Store  0.00
4  Latin American Restaurant  0.00


----M1H----
                  venue  freq
0                Bakery  0.12
1                  Bank  0.12
2       Thai Restaurant  0.12
3    Athletics & Sports  0.12
4  Caribbean R

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
um_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Postal Code'] = scarborough_grouped['Postal Code']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Gym
1,M1C,Bar,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym
2,M1E,Intersection,Bank,Restaurant,Rental Car Location,Breakfast Spot
3,M1G,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Convenience Store,Gym
4,M1H,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank
5,M1J,Playground,Vietnamese Restaurant,College Stadium,Gym,Grocery Store
6,M1K,Hobby Shop,Department Store,Discount Store,Chinese Restaurant,Coffee Shop
7,M1L,Bakery,Ice Cream Shop,Bus Line,Intersection,Metro Station
8,M1M,American Restaurant,Motel,Hobby Shop,Gym,Grocery Store
9,M1N,College Stadium,Skating Rink,General Entertainment,Café,Vietnamese Restaurant


In [28]:
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:16]

array([1, 0, 1, 3, 1, 2, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1])

In [29]:
scarborough_merged = Scarborough[0:16]

# add clustering labels
scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Postal Code'), on='Postal Code')

scarborough_merged.head() # check the last columns!

<ipython-input-29-0629b5ab95bd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  scarborough_merged['Cluster Labels'] = kmeans.labels_


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Gym
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Bar,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Intersection,Bank,Restaurant,Rental Car Location,Breakfast Spot
22,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Convenience Store,Gym
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank


In [30]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.78, -79.20], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Postal Code'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### cluster1 

In [31]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0,Bar,Vietnamese Restaurant,Convenience Store,Hakka Restaurant,Gym


### cluster2

In [32]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Gym
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1,Intersection,Bank,Restaurant,Rental Car Location,Breakfast Spot
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1,Hobby Shop,Department Store,Discount Store,Chinese Restaurant,Coffee Shop
44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,1,Bakery,Ice Cream Shop,Bus Line,Intersection,Metro Station
58,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,1,College Stadium,Skating Rink,General Entertainment,Café,Vietnamese Restaurant
65,M1P,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",43.757410,-79.273304,1,Indian Restaurant,Vietnamese Restaurant,Chinese Restaurant,Pet Store,Convenience Store
71,M1R,Scarborough,"Wexford, Maryvale",43.750072,-79.295849,1,Auto Garage,Bakery,Sandwich Place,Middle Eastern Restaurant,Vietnamese Restaurant
78,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Latin American Restaurant,Breakfast Spot,Print Shop,Lounge,Vietnamese Restaurant
82,M1T,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",43.781638,-79.304302,1,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Pharmacy,Italian Restaurant


In [33]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
32,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Playground,Vietnamese Restaurant,College Stadium,Gym,Grocery Store


### cluster4

In [34]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean BBQ Restaurant,Vietnamese Restaurant,Convenience Store,Gym


### cluster5

In [35]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
51,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,4,American Restaurant,Motel,Hobby Shop,Gym,Grocery Store
